In [2]:
from itertools import product
import pandas as pd

class Jeu:
    def __init__(self, joueurs,stategies, profil,utilites):
        self.joueurs = joueurs
        self.nb_joueurs = len(joueurs)
        self.strategies = stategies
        self.profil = profil
        self.utility = utilites

    def allStrategyProfils(self):
        allStrategy = []
        for i in range(len(self.strategies)):
            allStrategy.append(self.strategies[i][1])
        allStrategy =  list(product(*allStrategy))
        allStrategy2 = []
        for(i,elem) in enumerate(allStrategy):
            stat = []
            for joueur in self.joueurs:
                stat.append((joueur,elem[self.joueurs.index(joueur)]))
            allStrategy2.append(stat)
        return allStrategy2

    def removePlayer(self, joueur, profil):
        profil_copy = profil.copy()
        profil_copy = [x for x in profil_copy if x[0] != joueur] # On supprime le joueur du profil
        return profil_copy


    def addPlayer(self,player,strategy,profil):
           profil_copy = profil.copy()
           if(player,strategy) not in profil_copy:
                for i in range(len(profil_copy)):
                    if(profil_copy[i][0] == player):
                        profil_copy[i] = (player,strategy) # On remplace la stratégie du joueur par la nouvelle
           return profil_copy

    def isDominated(self,player,strategy1):
        strategies_joueurs = self.strategies[self.joueurs.index(player)][1]
        for strategy2 in strategies_joueurs:
            if(strategy1 != strategy2):
                if(self.isDominated2(player,strategy1,strategy2)): # On regarde si la stratégie est dominée par une autre
                    return True
        return False

    def isDominated2(self,player,strategy1,strategy2):
        allProfils = self.allStrategyProfils()
        for profil in allProfils:
            profilStrat1 = profil.copy()
            profilStrat2 = profil.copy()

            profilStrat1 = self.addPlayer(player,strategy1, profil) # On remplace la stratégie du joueur par la nouvelle
            profilStrat2 = self.addPlayer(player,strategy2,profil) # On remplace la stratégie du joueur par la nouvelle

            if(self.utility[player][str(profilStrat1)] >= self.utility[player][str(profilStrat2)]): # Si la nouvelle stratégie est moins bonne
                return False
        return True


    def IESDS(self):
        changed = True
        while(changed):
            changed = False
            for player in self.joueurs:
                for strategy in self.strategies[self.joueurs.index(player)][1]: # Pour chaque stratégie du joueur
                    if(self.isDominated(player,strategy)): # Si la stratégie est dominée
                        print("Strategie " + strategy + " de " + player + " est dominée")
                        self.strategies[self.joueurs.index(player)][1].remove(strategy) # On la supprime
                        changed = True
        return self.strategies # On retourne les stratégies restantes

    def securityLevel(self,player,strategy):
        mini = 1e10 # On initialise la valeur minimale à une valeur très grande
        allProfils = self.allStrategyProfils()
        for profil in allProfils: # Pour chaque profil
            if((player,strategy) in profil): # Si la stratégie est dans le profil
                val = self.utility[player][str(profil)] # On récupère l'utilité du joueur
                if(val < mini): # Si l'utilité est plus petite que la valeur minimale
                    mini = val # On met à jour la valeur minimale
        return mini
    
    def isParetoDominant(self,profil1,profil2):
        for joueur in self.joueurs: # Pour chaque joueur
            if(self.utility[joueur][str(profil1)] <= self.utility[joueur][str(profil2)]) :  # Si l'utilité du joueur est plus petite dans le profil 1
                return False # Le profil n'est pas pareto dominant
        return True
    
    def optimumPareto(self):
        allProfils = self.allStrategyProfils()
        pareto = []
        for profil in allProfils:
            for profil2 in allProfils:
                # On regarde si le profil est pareto dominant par rapport à tous les autres profils
                if(profil != profil2): # (on ne compare pas le profil avec lui même)
                    isPareto = self.isParetoDominant(profil,profil2)
            if(isPareto): # Si le profil est pareto dominant
                pareto.append(profil) # On l'ajoute à la liste des profils de pareto
        return pareto
    
    def isNash(self,profil1):
        allProfils = self.allStrategyProfils()
        for strategie in profil1: # Pour chaque stratégie du profil
            strategie_adv_fixe = profil1.copy()
            strategie_adv_fixe.remove(strategie)
            strategie_adv_fixe = strategie_adv_fixe[0] # On fixe la stratégie de l'adversaire
            for profil2 in allProfils: # Pour chaque profil
                if(profil1 != profil2 and strategie_adv_fixe in profil2): # Si le profil est différent du profil de base et que la stratégie de l'adversaire est la même
                   if(self.utility[strategie[0]][str(profil1)] < self.utility[strategie[0]][str(profil2)]): # Si l'utilité du joueur est plus petite dans le profil 1
                       return False # Le profil n'est pas un équilibre de Nash
        return True                     
                   
    
    def equilibreNash(self):
        allProfils = self.allStrategyProfils()
        nash=[]
        isNash = False
        for profil in allProfils: # Pour chaque profil
            isNash = False
            isNash = self.isNash(profil) # On regarde si le profil est un équilibre de Nash
            if(isNash): # Si le profil l'est
                nash.append(profil) # On l'ajoute à la liste
        return nash
    
    def affichageJeu(self):
        # Créer un DataFrame vide
        df = pd.DataFrame(index=self.strategies[0][1], columns=self.strategies[1][1])
        allProfils = self.allStrategyProfils()
        for profil in allProfils:
            l = [profil[0][1],profil[1][1]]
            df.loc[l[0],l[1]] = [self.utility[profil[0][0]][str(profil)],self.utility[profil[1][0]][str(profil)]]
        
        df.loc[self.joueurs[0], :] = ""
        df.loc[:, self.joueurs[1]] = ""
        return df


# Q.1 Elimination des stratégies dominées (IESDS)

In [3]:
joueurs = ["Joueur1","Joueur2"]
strategies = [("Joueur1",["D","E","F"]),("Joueur2",["A","B","C"])]

utilites = []
utilitesJ1 = {str([("Joueur1", "D"), ("Joueur2", "A")]) : 4,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 5,
                str([("Joueur1", "D"), ("Joueur2", "C")]) : 6,
                str([("Joueur1", "E"), ("Joueur2", "A")]) : 2,
                str([("Joueur1", "E"), ("Joueur2", "B")]) : 8,
                str([("Joueur1", "E"), ("Joueur2", "C")]) : 3,
                str([("Joueur1", "F"), ("Joueur2", "A")]) : 3,
                str([("Joueur1", "F"), ("Joueur2", "B")]) : 9,
                str([("Joueur1", "F"), ("Joueur2", "C")]) : 2}

utilitesJ2 = {str([("Joueur1", "D"), ("Joueur2", "A")]) : 3,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 1,
                str([("Joueur1", "D"), ("Joueur2", "C")]) : 2,
                str([("Joueur1", "E"), ("Joueur2", "A")]) : 1,
                str([("Joueur1", "E"), ("Joueur2", "B")]) : 4,
                str([("Joueur1", "E"), ("Joueur2", "C")]) : 6,
                str([("Joueur1", "F"), ("Joueur2", "A")]) : 0,
                str([("Joueur1", "F"), ("Joueur2", "B")]) : 6,
                str([("Joueur1", "F"), ("Joueur2", "C")]) : 8}

utilites = {"Joueur1" : utilitesJ1, "Joueur2" : utilitesJ2}

j = Jeu(joueurs,strategies,strategies,utilites)
print(j.affichageJeu())
print("Les stratégies dominées sont :",j.IESDS())

              A       B       C Joueur2
D        [4, 3]  [5, 1]  [6, 2]        
E        [2, 1]  [8, 4]  [3, 6]        
F        [3, 0]  [9, 6]  [2, 8]        
Joueur1                                
Strategie B de Joueur2 est dominée
Strategie E de Joueur1 est dominée
Strategie F de Joueur1 est dominée
Strategie C de Joueur2 est dominée
Les stratégies dominées sont : [('Joueur1', ['D']), ('Joueur2', ['A'])]


# Q.2 Niveau de sécurité

In [4]:
joueurs = ["Joueur1","Joueur2"]
strategies = [("Joueur1",["C","D"]),("Joueur2",["A","B"])]

utilites = []
utilitesJ1 = {str([("Joueur1", "C"), ("Joueur2", "A")]) : 9,
                str([("Joueur1", "C"), ("Joueur2", "B")]) : 0,
                str([("Joueur1", "D"), ("Joueur2", "A")]) : 8,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 7}

utilitesJ2 = {str([("Joueur1", "C"), ("Joueur2", "A")]) : 9,
                str([("Joueur1", "C"), ("Joueur2", "B")]) : 8,
                str([("Joueur1", "D"), ("Joueur2", "A")]) : 0,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 7}

utilites = {"Joueur1" : utilitesJ1, "Joueur2" : utilitesJ2}

j = Jeu(joueurs,strategies,strategies,utilites)
print(j.affichageJeu())
print("Le joueur1 a un security level de  " + str(j.securityLevel("Joueur1","D")))

              A       B Joueur2
C        [9, 9]  [0, 8]        
D        [8, 0]  [7, 7]        
Joueur1                        
Le joueur1 a un security level de  7


# Q.3 Les optima de Pareto

In [5]:
joueurs = ["Joueur1","Joueur2"]
strategies = [("Joueur1",["C","D"]),("Joueur2",["A","B"])]

utilites = []
utilitesJ1 = {str([("Joueur1", "C"), ("Joueur2", "A")]) : 4,
                str([("Joueur1", "C"), ("Joueur2", "B")]) : 0,
                str([("Joueur1", "D"), ("Joueur2", "A")]) : 3,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 4}

utilitesJ2 = {str([("Joueur1", "C"), ("Joueur2", "A")]) : 4,
                str([("Joueur1", "C"), ("Joueur2", "B")]) : 3,
                str([("Joueur1", "D"), ("Joueur2", "A")]) : 1,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 5}

utilites = {"Joueur1" : utilitesJ1, "Joueur2" : utilitesJ2}

j = Jeu(joueurs,strategies,strategies,utilites)
print(j.affichageJeu())
print("Les optima de Pareto sont :",j.optimumPareto())

              A       B Joueur2
C        [4, 4]  [0, 3]        
D        [3, 1]  [4, 5]        
Joueur1                        
Les optima de Pareto sont : [[('Joueur1', 'D'), ('Joueur2', 'B')]]


# Q.4 Les équilibres de Nash

In [6]:
joueurs = ["Joueur1","Joueur2"]
strategies = [("Joueur1",["D","E","F"]),("Joueur2",["A","B","C"])]

utilites = []
utilitesJ1 = {str([("Joueur1", "D"), ("Joueur2", "A")]) : 4,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 5,
                str([("Joueur1", "D"), ("Joueur2", "C")]) : 6,
                str([("Joueur1", "E"), ("Joueur2", "A")]) : 2,
                str([("Joueur1", "E"), ("Joueur2", "B")]) : 8,
                str([("Joueur1", "E"), ("Joueur2", "C")]) : 3,
                str([("Joueur1", "F"), ("Joueur2", "A")]) : 3,
                str([("Joueur1", "F"), ("Joueur2", "B")]) : 9,
                str([("Joueur1", "F"), ("Joueur2", "C")]) : 2}

utilitesJ2 = {str([("Joueur1", "D"), ("Joueur2", "A")]) : 3,
                str([("Joueur1", "D"), ("Joueur2", "B")]) : 1,
                str([("Joueur1", "D"), ("Joueur2", "C")]) : 2,
                str([("Joueur1", "E"), ("Joueur2", "A")]) : 1,
                str([("Joueur1", "E"), ("Joueur2", "B")]) : 4,
                str([("Joueur1", "E"), ("Joueur2", "C")]) : 6,
                str([("Joueur1", "F"), ("Joueur2", "A")]) : 0,
                str([("Joueur1", "F"), ("Joueur2", "B")]) : 6,
                str([("Joueur1", "F"), ("Joueur2", "C")]) : 8}

utilites = {"Joueur1" : utilitesJ1, "Joueur2" : utilitesJ2}

j = Jeu(joueurs,strategies,strategies,utilites)
print(j.affichageJeu())
print("Les équilibres de Nash sont :",j.equilibreNash())

              A       B       C Joueur2
D        [4, 3]  [5, 1]  [6, 2]        
E        [2, 1]  [8, 4]  [3, 6]        
F        [3, 0]  [9, 6]  [2, 8]        
Joueur1                                
Les équilibres de Nash sont : [[('Joueur1', 'D'), ('Joueur2', 'A')]]
